<a href="https://colab.research.google.com/github/firststef/APLN-WSD/blob/master/WSD_Romanian_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from bs4 import BeautifulSoup

def parse_xml(path):
  with open(path, encoding='windows-1252') as fp:
    soup = BeautifulSoup(fp, 'html.parser')
  data = soup.find_all('instance')
  new_data = []
  for inst in data:
    sentence = inst.find('context').decode_contents().strip('\n')
    a, *b = sentence.split('<head>')
    b, *c = ''.join(b).split('</head>')
    sentence = a + b.lower() + ''.join(c)
    new_data.append((sentence, inst.attrs['id'], b.lower()))
  return new_data

train_se3 = parse_xml("/content/drive/MyDrive/Colab Notebooks/wsd/RomanianLS.train")
test_se3 = parse_xml("/content/drive/MyDrive/Colab Notebooks/wsd/RomanianLS.test")

In [ ]:
import os
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

ro_stop_words = stopwords.words('romanian')

def import_data(data):
  x = []
  y = []
  for entry in data:
    x.append(entry[0])
    y.append(entry[1])
  return [x, y]

x, y = import_data(train_se3)

tfidfvectorizer = TfidfVectorizer(strip_accents="ascii", stop_words=ro_stop_words, token_pattern=r"(?u)\b[A-Za-z]+\b")
x = tfidfvectorizer.fit_transform(x).toarray()

classifier = RandomForestClassifier(n_jobs=1, max_depth=1000, max_features=10000)
classifier.fit(x, y)

with open("classifier.bin", "wb") as f:
    pickle.dump(classifier, f)
with open("vectorizer.bin", "wb") as f:
    pickle.dump(tfidfvectorizer, f)

y_prediction = classifier.predict(x)

print(confusion_matrix(y, y_prediction))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import sys
import os
import pickle

def get_classifier():
  f_classifier = open("classifier.bin", "rb")
  classifier = pickle.load(f_classifier)
  f_classifier.close()
  return classifier

def get_vectorizer():
  f_vectorizer = open("vectorizer.bin", "rb")
  vectorizer = pickle.load(f_vectorizer)
  f_vectorizer.close()
  return vectorizer

x, y = import_data(test_se3)

def scan(folder, output_file, debug=False):
  classifier = get_classifier()
  vectorizer = get_vectorizer()

  converted = vectorizer.transform(x)
  verdicts = classifier.predict(converted)
  correct = 0
  for i in range(0, len(verdicts)):
    if verdicts[i] == y[i]:
      correct += 1
  print(correct / len(verdicts))
